## Практическое задание

<ol>
    <li>Попробовать улучшить точность распознования образов cifar 10 сверточной нейронной сетью, рассмотренной на уроке. Приложить анализ с описанием того, что улучшает работу нейронной сети и что ухудшает.
    </li>
    <li>Описать также в анализе какие необоходимо внести изменения  в получившуюся у вас нейронную сеть если бы ей нужно было работать не с cifar10, а с MNIST, CIFAR100 и IMAGENET.
    </li>
</ol>

In [6]:
#pip install tensorflow

In [7]:
from __future__ import print_function
import keras
from keras.applications import ResNet50
from tensorflow.keras.datasets import cifar10
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dense, GlobalAveragePooling2D
import os

In [8]:
#Загрузка данных cifar10
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

In [9]:
# Установка параметров нейросети
batch_size = 64
num_classes = 1000
epochs = 10

# Создание объекта ImageDataGenerator для аугментации данных и нормализация данных
datagen = ImageDataGenerator(rescale=1.0/255.0)

# Создание генераторов для обучающей и тестовой выборок
train_generator = datagen.flow(x_train, y_train, batch_size=batch_size)
test_generator = datagen.flow(x_test, y_test, batch_size=batch_size)

In [ ]:
# Создание предварительно обученной модели ResNet50
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Создание модели для Featurization, рагружаем базовую модель в последовательную
model = Sequential()
model.add(base_model)
model.add(GlobalAveragePooling2D())
model.add(Dense(num_classes, activation='softmax'))

# Компиляция модели
model.compile(loss='sparse_categorical_crossentropy',
              optimizer='SGD',
              metrics=['accuracy'])

# Обучение модели
model.fit_generator(train_generator,
                    steps_per_epoch=len(x_train) // batch_size,
                    epochs=epochs,
                    validation_data=test_generator,
                    validation_steps=len(x_test) // batch_size)

# Оценка производительности модели на тестовых данных
score = model.evaluate_generator(test_generator, steps=len(x_test) // batch_size)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

C:\Users\lisat\AppData\Local\Temp\ipykernel_16328\2084830545.py:16: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  model.fit_generator(train_generator,


Epoch 1/10
781/781 [==============================] - 1707s 2s/step - loss: 1.2899 - accuracy: 0.1001 - val_loss: 2.2939 - val_accuracy: 0.0516
Epoch 2/10
781/781 [==============================] - 1538s 2s/step - loss: 0.6534 - accuracy: 0.1016 - val_loss: 0.7631 - val_accuracy: 0.0961
Epoch 3/10
781/781 [==============================] - 1520s 2s/step - loss: 0.4415 - accuracy: 0.1021 - val_loss: 0.6914 - val_accuracy: 0.0986
Epoch 4/10
105/781 [===>..........................] - ETA: 21:48 - loss: 0.2743 - accuracy: 0.1001

Исходя из анализа, можно сделать вывод о сложности выработки четких правил установки соотношения параметров для достижения наилучших показателей при использовании нейросетей. Также, можно сказать, что сети ResNet50 и ResNet50V2, по всей видимости, не очень подходят для обработки датасета Cifar10 (у обеих схожие результаты на низком уровне). Лучшие показатели, тем не менее, были обеспечены при использовании функции loss - sparse_categorical_crossentropy (без данной функции модель вообще ничего не могла классифицировать), optimizer - SGD и batch_size = 64. Также сеть фактически не обучалась и метрика accuracy колебалась на протяжении 25 эпох (при различных параметрах сети) от 9 до 15 % и, по сути, зависела от конкретного фрагмента датасета направляемого на тест.